# Hyperparameter Tuning using HyperDrive

## Dependencies

In [1]:
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

from azureml.widgets import RunDetails

from sklearn import datasets
from azureml.train.sklearn import SKLearn

import pandas as pd
import numpy as np
import os

## Dataset

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [3]:
url = "https://raw.githubusercontent.com/mirsadraee/Udacity_ND_Azure_Machine_Learning_Projects/develop/Project_03/heart_failure_clinical_records_dataset.csv"

dataset = TabularDatasetFactory.from_delimited_files(url)

key = "heart-failure-prediction"
description_text = "Prediction of heart failure - NDAML"

dataset = dataset.register(workspace=ws,
                           name=key,
                           description=description_text)
dataset = dataset.to_pandas_dataframe()

dataset.head()

x = dataset.drop('DEATH_EVENT', axis=1)
y = dataset[['DEATH_EVENT']]

In [4]:
# Choose a name for your CPU cluster
cluster_name = "compute-cluster-p3"

# Create compute cluster
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('An existing cluster will be used!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print('An new cluster will be used now!')

aml_compute.wait_for_completion(show_output=True)

print(aml_compute.get_status().serialize())


An existing cluster will be used!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-04-22T13:27:17.204000+00:00', 'errors': None, 'creationTime': '2023-04-22T12:10:29.432666+00:00', 'modifiedTime': '2023-04-22T12:10:36.059749+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, delay_evaluation = 2)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.01,0.1,1,10,100,500,1000),
        '--max_iter': choice(1,10,50,100,200)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# est = ScriptRunConfig(source_directory='.',
#                      command=['python', 'train.py'],
#                      compute_target=aml_compute,
#                      environment=sklearn_env)

est = SKLearn(source_directory = "./",
            compute_target=aml_compute,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cc702253-7609-4de7-a36d-952100a1ad8b
Web View: https://ml.azure.com/runs/HD_cc702253-7609-4de7-a36d-952100a1ad8b?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-231701/workspaces/quick-starts-ws-231701&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-04-22T13:48:50.472971][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-22T13:48:50.9579966Z][SCHEDULER][INFO]Scheduling job, id='HD_cc702253-7609-4de7-a36d-952100a1ad8b_0' 
[2023-04-22T13:48:51.0805229Z][SCHEDULER][INFO]Scheduling job, id='HD_cc702253-7609-4de7-a36d-952100a1ad8b_1' 
[2023-04-22T13:48:51.1910274Z][SCHEDULER][INFO]Scheduling job, id='HD_cc702253-7609-4de7-a36d-952100a1ad8b_2' 
[2023-04-22T13:48:51.231399][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-22T13:48:51.2692994Z][SCHEDULER][INFO]Scheduling job, id='HD_cc702253-7609-4de7-a36d-952100a1ad

{'runId': 'HD_cc702253-7609-4de7-a36d-952100a1ad8b',
 'target': 'compute-cluster-p3',
 'status': 'Completed',
 'startTimeUtc': '2023-04-22T13:48:49.714471Z',
 'endTimeUtc': '2023-04-22T14:03:56.302836Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '04986c97-0063-4cf0-be5a-3f3bd00a5a0d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '35',
  'score': '0.8333333333333334',
  'best_child_run_id': 'HD_cc702253-7609-4de7-a36d-952100a1ad8b_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_cc702253-7609-4de7-a36d-952100a1ad8b_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'aml

## Best Model

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive_experiment,HD_cc702253-7609-4de7-a36d-952100a1ad8b_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run_metrics = best_run.get_metrics()
best_run_metrics 

{'Regularization Strength:': 100.0,
 'Max iterations:': 50,
 'Accuracy': 0.8333333333333334}

In [10]:
# Save the best model
best_run.register_model(model_name='Model_Hyperdive_best_run.pkl', model_path='./') 

# Download the model file
best_run.download_file('outputs/model.pkl', 'hyperdrive_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- [X]  I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- [X]  I have taken a screenshot showing the model endpoint as active.
- [X]  The project includes a file containing the environment details.

